# load data

In [1]:
%run ./function.ipynb

In [2]:
PATH = 'D:/Temp/NCTU/crystal/第二階段/new_alert/日數分類_營業日/'
FILE = 'TWN_A11_01_day15_with_aggregated_txn_scenario_txn_type_key_營業日.csv'
df = pd.read_csv(PATH + FILE)
df.columns

Index(['alert_id', 'run_date', 'primary_entity_level_code',
       'primary_entity_number', 'scenario_name', 'TO_SAR', 'Acct_No',
       'Cust_No', 'Cust_Key', 'Cust_Segmentation', 'Cust_Typ_Cd', 'Cust_Id_Id',
       'Cust_Id_Typ_Desc', 'Cust_Id_State_Cd', 'Cust_Dt_Of_Birth',
       'Cust_Sts_Desc', 'Street_Postal_Cd', 'Street_Cntry_Cd',
       'Mailing_Postal_Cd', 'Mailing_Cntry_Cd', 'Residence_Cntry_Cd',
       'Citizenship_Cntry_Cd', 'Org_Cntry_Of_Buz_Cd', 'Empl_No', 'Ocup_Code',
       'job_title_code', 'Lst_Cont_Dt', 'Politically_Exposed_Pers_Ind',
       'Non_Prft_Org_Ind', 'Cust_Since_Dt', 'Lst_Susp_Actv_Rpt_Dt',
       'Extnl_Cust_Ind', 'Negative_News_Ind', 'Prim_Br_No', 'Obu_Ind',
       'Riskclass_Cd', 'risk_score', 'last_risk_assessment_date',
       'change_current_ind', 'change_begin_date', 'change_end_date',
       'Credit_Amt', 'number_of_Credit', 'Debit_Amt', 'number_of_Debit'],
      dtype='object')

In [4]:
df['TO_SAR'] = df['TO_SAR'].replace(np.nan, 0)
df['TO_SAR'] = df['TO_SAR'].replace('F', 1)
df['TO_SAR'] = df['TO_SAR'].replace('NF', 1)

In [5]:
df['TO_SAR'].value_counts()

0    86324
1      446
Name: TO_SAR, dtype: int64

In [6]:
SAR_type = 'TO_SAR'
train_set, test_set = ReadFile(df, SAR_type)

Verify on time series
shape of data: (86770, 39)
total number of TO_SAR: 446
********************************
time interval in train set:2019-03-04~2020-04-29
TO_SAR in train set: 267
TO_SAR/Total in train set 0.005128599143312653
split dataset to train, test...
********************************
time interval in test set:2020-02-19~2020-12-31
TO_SAR in test set: 179
TO_SAR/Total in test set 0.0051571638479933155
********************************
train data shape:  (52061, 38)
test data shape:  (34709, 38)


In [7]:
train_set = train_set.dropna(subset= ['Credit_Amt', 'number_of_Credit', 'Debit_Amt', 'number_of_Debit'])
test_set = test_set.dropna(subset= ['Credit_Amt', 'number_of_Credit', 'Debit_Amt', 'number_of_Debit'])

In [8]:
SAR = train_set[train_set.TO_SAR!= 0]
Non_SAR = train_set[train_set.TO_SAR== 0]

# function

In [9]:
def intialPoint(feature, data):
    return data[feature][np.random.randint(0, len(data))]

In [10]:
def conditionalEntropy(new_sam_0, new_sam_1):
    total = len(new_sam_1) + len(new_sam_0)
    return Entropy(new_sam_0, SAR_type) * (len(new_sam_0) / total) \
            + Entropy(new_sam_1, SAR_type) * (len(new_sam_1) / total)

In [11]:
def informationGain(x_thre, y_thre, x_name, y_name, data):
    
    idx1 = set(data[data[x_name] >= x_thre].index)
    idx2 = set(data[data[y_name] >= y_thre].index)
    
    #idx3 = set(data[data[y_name] >= float(y_thre)].index)
    
    idx = list(idx1 & idx2)
    new_sam_1 = data.iloc[idx]
    new_sam_0 = data.drop(index= new_sam_1.index)
    
    if len(set(new_sam_1.index) & set(new_sam_0.index)) != 0:
        raise ValueError('new sam 0 and new sam 1 are overlapped!')
        
    return -(father_entropy - conditionalEntropy(new_sam_0, new_sam_1))

In [12]:
def recallFilterRate(x_thre, y_thre, x_name, y_name, data):
    idx1 = set(data[data[x_name] > x_thre].index)
    idx2 = set(data[data[y_name] > y_thre].index)
    
    idx = list(idx1 & idx2)
    
    new_sam_1 = data.iloc[idx]
    new_sam_0 = data.drop(index= new_sam_1.index)
    
    if len(set(new_sam_1.index) & set(new_sam_0.index)) != 0:
        raise ValueError('new_sam_1 and new_sam_0 are voerlapped')
        
    TP = new_sam_1[new_sam_1[SAR_type] != 0].shape[0]
    TN = new_sam_0[new_sam_0[SAR_type] == 0].shape[0]
    FP = new_sam_1[new_sam_1[SAR_type] == 0].shape[0]
    FN = new_sam_0[new_sam_0[SAR_type] != 0].shape[0]
    
    recall = TP / (TP + FN)
    filter_rate = (TN + FN) / (TP + TN + FP + FN)
    
    return recall, filter_rate

# Newton Method

## original version
2022/10/12 加入 backtracking line search

In [12]:
def finiteDifferenceIG(x1, y1, x_name, y_name, data):
    x2 = data.loc[data[x_name] > x1, x_name].reset_index(drop= True)[90]
    y2 = data.loc[data[y_name] > y1, y_name].reset_index(drop= True)[0]
    print(f'x1:{x1}, y1:{y1}')
    print(f'x2:{x2}, y2:{y2}')
    delta_x = x2 - x1
    delta_y = y2 - y1
    
    z = informationGain(x1, y1, x_name, y_name, data)
    z_delta_x = informationGain(x1 + delta_x, y1, x_name, y_name, data)
    z_delta_y = informationGain(x1, y1 + delta_y, x_name, y_name, data)
    z_delta_2x = informationGain(x1 + delta_x * 2, y1, x_name, y_name, data)
    z_delta_2y = informationGain(x1, y1 + delta_y * 2, x_name, y_name, data)
    z_delta_xy = informationGain(x1 + delta_x, y1 + delta_y, x_name, y_name, data)
    
    grad = np.array([(z_delta_x - z) / delta_x, (z_delta_y - z) / delta_y])
    hessian = np.array([[(z_delta_2x - z_delta_x * 2 + z) / (delta_x * delta_x), (z_delta_xy - z_delta_x - z_delta_y + z) / (delta_x * delta_y)], \
                       [(z_delta_xy - z_delta_x - z_delta_y + z) / (delta_x * delta_y), (z_delta_2y - z_delta_y * 2 + z) / (delta_y * delta_y)]])
    
    return grad, hessian

In [13]:
def Newton(x1, y1, x_name, y_name, data):
    results = pd.DataFrame(columns= ['iteration number', 'x1', 'y1', 'new x1', 'new y1', 'information gain', 'recall', 'filter rate'])
    n = 1
    while n < N_max:
        grads, hessian = finiteDifferenceIG(x1, y1, x_name, y_name, data)
        print(f'gradient: {grads}')
        print(f'hessian matrix:\n{hessian}')
        
        if np.linalg.det(hessian) == 0:
            raise ValueError('hessian matirx is singular matrix, can not invertible')
        
        temp_point = np.array([[x1], [y1]]) - step_size * np.linalg.inv(hessian) @ grads
    
        information_gain = informationGain(new_points[0], new_points[1], x_name, y_name, data)
    
        recall, filter_rate = recallFilterRate(new_points[0], new_points[1], x_name, y_name, data)
        results = results.append({'iteration number': n, 'x1':x1, 'y1':y1, 'new x1': new_points[0], 'new y1': new_points[1], 'information gain':information_gain, \
                                  'recall':recall, 'filter rate': filter_rate}, ignore_index= True)
    
        if np.linalg.norm(grads, 2) < eplison:
            print('Early stopping' + '------' * 8 + f'\niteration number:{n} new x1:{new_points[0]}, new y1:{new_points[1]}')
            break
            
            
        x1 = new_points[0]
        y1 = new_points[1]
        n += 1
    
    print('Finish' + '-----' * 8 + f'\niteration number:{n} new x1: {new_points[0]}, new y1: {new_points[1]}')
    
    return results

In [14]:
#hyperparameter
SAR_type = 'TO_SAR'
eplison = 1e-10
N_max = 1000
x_name = 'Credit_Amt'
y_name = 'number_of_Credit'
father_entropy = Entropy(train_set, SAR_type)

In [134]:
if __name__ == '__main__':
    np.random.seed(1022)
    x1 = intialPoint('Credit_Amt', train_set)
    y1 = intialPoint('number_of_Credit', train_set)
    #print(f'x1:{x1}, y1:{y1}')
    results = Newton(x1, y1, x_name, y_name, train_set)

x1:0.0, y1:0.0
x2:100000.0, y2:1.0
gradient: [8.20844399e-09 8.30795366e-04]
hessian matrix:
[[-9.19043979e-14 -8.30795366e-09]
 [-8.30795366e-09 -1.00183186e-03]]
x1:57320.324679250654, y1:0.3539324101496293
x2:172300.0, y2:1.0
gradient: [-6.1026367e-10  0.0000000e+00]
hessian matrix:
[[ 1.18154527e-15  0.00000000e+00]
 [ 0.00000000e+00 -2.92725679e-04]]
x1:573816.5466547816, y1:0.3539324101496293
x2:1010000.0, y2:1.0
gradient: [8.51692999e-11 0.00000000e+00]
hessian matrix:
[[-2.35000646e-15  0.00000000e+00]
 [ 0.00000000e+00 -7.32775801e-05]]
Early stopping------------------------------------------------
new x1: 610058.7023463736, new y1: 0.3539324101496293
Finish----------------------------------------
new x1: 610058.7023463736, new y1: 0.3539324101496293


In [126]:
results.tail(10)

,x1,y1,new x1,new y1,information gain,recall,filter rate
989,224704.739396,62.424185,224704.739396,62.758196,0.000006,0.007491,0.011448
990,224704.739396,62.758196,224704.739396,63.087476,0.000004,0.007491,0.010853
991,224704.739396,63.087476,224704.739396,62.391699,0.000006,0.007491,0.011448
992,224704.739396,62.391699,224704.739396,62.726171,0.000006,0.007491,0.011448
993,224704.739396,62.726171,224704.739396,63.055904,0.000004,0.007491,0.010853
994,224704.739396,63.055904,224704.739396,62.359168,0.000006,0.007491,0.011448
995,224704.739396,62.359168,224704.739396,62.694101,0.000006,0.007491,0.011448
996,224704.739396,62.694101,224704.739396,63.024288,0.000004,0.007491,0.010853
997,224704.739396,63.024288,224704.739396,62.326593,0.000006,0.007491,0.011448
998,224704.739396,62.326593,224704.739396,62.661987,0.000006,0.007491,0.011448


## new BFGS
2022/10/19

In [13]:
def gradientIG(point, x_name, y_name, data):
    x1 = float(point[0])
    y1 = float(point[1])

    if len(np.sort(np.unique(data.loc[data[x_name] > x1, x_name]), kind= 'stable')) > auxiliary_point_select_behind_x:
        x2 = np.sort(np.unique(data.loc[data[x_name] > x1, x_name]), kind= 'stable')[auxiliary_point_select_behind_x]
    else:
        x2 = np.sort(np.unique(data.loc[data[x_name] < x1, x_name]), kind= 'stable')[-auxiliary_point_select_behind_x]
    
    if len(np.sort(np.unique(data.loc[data[y_name] > y1, x_name]), kind= 'stable')) > auxiliary_point_select_behind_x:
        y2 = np.sort(np.unique(data.loc[data[y_name] > y1, x_name]), kind= 'stable')[auxiliary_point_select_behind_x]
    else:
        y2 = np.sort(np.unique(data.loc[data[y_name] < y1, x_name]), kind= 'stable')[-auxiliary_point_select_behind_x]
    
    delta_x = x2 - x1
    delta_y = y2 - y1
    
    z = informationGain(point, x_name, y_name, data)
    z_delta_x = informationGain(np.array([[x2], [y1]]), x_name, y_name, data)
    z_delta_y = informationGain(np.array([[x1], [y2]]), x_name, y_name, data)
    
    grad = np.array([[(z_delta_x - z) / delta_x], [(z_delta_y - z) / delta_y]])
    return grad

In [14]:
def informationGain(point, x_name, y_name, data):
    
    idx1 = set(data[data[x_name] >= float(point[0])].index)
    idx2 = set(data[data[y_name] >= float(point[1])].index)
    
    #idx3 = set(data[data[y_name] >= float(y_thre)].index)
    idx = list(idx1 & idx2)
    new_sam_1 = data.iloc[idx]
    new_sam_0 = data.drop(index= new_sam_1.index)
    
    if len(set(new_sam_1.index) & set(new_sam_0.index)) != 0:
        raise ValueError('new sam 0 and new sam 1 are overlapped')
        
    return -(father_entropy - conditionalEntropy(new_sam_0, new_sam_1))

In [15]:
def auxiliaryPoint(point, feature, number, data):
    delta = (data[feature].max() - data[feature].min()) / number
    
    return point + delta

In [16]:
def lineSearch(point, update_direction, grads, data):
    a = 1
    c1 = 1e-4
    c2 = 0.9
    z = informationGain(point, x_name, y_name, data)
    new_point = point + a * update_direction
    new_grads = gradientIG(point, x_name, y_name, data)
    new_z = informationGain(new_point, x_name, y_name, data)
    print(f'before loop')
    while (new_z >= z + (c1 * a * grads.T @ update_direction)) or (new_grads.T @ update_direction <= c2 * (grads.T @ update_direction)):
        a *= 0.5
        new_point = point + a * update_direction
        new_grads = gradientIG(new_point, x_name, y_name, data)
        
    return a

In [17]:
def Recall(point, x_name, y_name, data):
    idx1 = set(data[data[x_name] > float(point[0])].index)
    idx2 = set(data[data[y_name] > float(point[1])].index)
    idx = list(idx1 & idx2)
    print(f'idx:{len(idx)}')
    new_sam_1 = data.iloc[idx]
    new_sam_0 = data.drop(index= new_sam_1.index)
    
    if len(set(new_sam_1.index) & set(new_sam_0.index)) != 0:
        raise ValueError('new sam 1 and new sam 0 are overlapped')
    
    TP = new_sam_1[new_sam_1[SAR_type] != 0].shape[0]
    TN = new_sam_0[new_sam_0[SAR_type] == 0].shape[0]
    FP = new_sam_1[new_sam_1[SAR_type] == 0].shape[0]
    FN = new_sam_0[new_sam_0[SAR_type] != 0].shape[0]
    
    return -(TP / (TP + FN))

In [18]:
def RecallFilterRate(point, x_name, y_name, data):
    idx1 = set(data[data[x_name] > float(point[0])].index)
    idx2 = set(data[data[y_name] > float(point[1])].index)
    
    idx = list(idx1 & idx2)
    
    new_sam_1 = data.iloc[idx]
    new_sam_0 = data.drop(index= new_sam_1.index)
    
    if len(set(new_sam_1.index) & set(new_sam_0.index)) != 0:
        raise ValueError('new_sam_1 and new_sam_0 are voerlapped')
        
    TP = new_sam_1[new_sam_1[SAR_type] != 0].shape[0]
    TN = new_sam_0[new_sam_0[SAR_type] == 0].shape[0]
    FP = new_sam_1[new_sam_1[SAR_type] == 0].shape[0]
    FN = new_sam_0[new_sam_0[SAR_type] != 0].shape[0]
    
    recall = TP / (TP + FN)
    filter_rate = (TP + FP) / (TP + TN + FP + FN)
    
    return recall, filter_rate

In [19]:
def BFGS(point0, x_name, y_name, data):
    results = pd.DataFrame(columns= ['iteration number', 'x1', 'y1', 'new x1', 'new y1', 'information gain', 'recall', 'filter rate'])
    d = 2
    grads = gradientIG(point0, x_name, y_name, data)
    H = np.eye(d)
    point = point0
    n = 1
    
    while np.linalg.norm(grads) > 1e-50:
        if n > N_max:
            print(f'Maximum iteration reached!')
            break
        
        
        update_direction = -H @ grads
        #step_size = lineSearch(point, update_direction, grads, data)
        print(f'update_direction:\n{update_direction}')
        new_point = point + (step_size * update_direction)
        new_grads = gradientIG(new_point, x_name, y_name, data)
        
        if np.isnan(new_grads).all() == True:
            break
    
        y = new_grads - grads
        s = new_point - point
        denominator = 1 / (y.T @ s)
        li = np.eye(d) - (denominator * (s @ y.T))
        ri = np.eye(d) - (denominator * (y @ s.T))
        H = li @ H @ ri + (denominator * (s @ s.T))
        grads = new_grads
        point = new_point
        
        information_gain = informationGain(point, x_name, y_name, data)
        recall, filter_rate = RecallFilterRate(point, x_name, y_name, data)
        
        print(f'n={n}, new x1:{point[0][0]}, new y1: {point[1][0]},\ninformation gain: {information_gain}, recall: {recall}, filter rate: {filter_rate}')
        
        results = results.append({'iteration number': n, 'x1': float(point[0]), 'y1': float(point[1]), 'new x1': float(new_point[0]), \
                                 'new y1': float(new_point[1]), 'information gain': information_gain, 'recall': recall, \
                                 'filter rate': filter_rate}, ignore_index= True)
        print('*'*32)
        print(f'new_grads:\n{new_grads}')
        print(f'hessian matrix:\n{H}')
        n += 1

    if np.linalg.norm(grads) < 1e-50:
        print('Early stopping!')
    
    return results

In [20]:
#hyperparameter
SAR_type = 'TO_SAR'
eps = 1e-1000
N_max = 100
x_name = 'Credit_Amt'
y_name = 'number_of_Credit'
#step_size = np.array([[1e12], [1e6]])
step_size = 1
auxiliary_point_select_behind_x = 3
auxiliary_point_select_behind_y = 3
father_entropy = Entropy(train_set, SAR_type)

In [23]:
if __name__ == '__main__':
    np.random.seed(928)
    x1 = intialPoint('Credit_Amt', train_set)
    y1 = intialPoint('number_of_Credit', train_set)
    print(f'initial  point x1:{x1}, y1:{y1}')
    point = np.array([[x1], [y1]])
    results = BFGS(point, x_name, y_name, train_set)

initial  point x1:537664.0, y1:20.0
update_direction:
[[-2.83161263e-11]
 [-6.07450306e-10]]
n=1, new x1:537664.0, new y1: 19.99999999939255,
information gain: -0.00020901150114722294, recall: 0.052434456928838954, filter rate: 0.12224356511717249
********************************
new_grads:
[[2.83161263e-11]
 [6.20144083e-10]]
hessian matrix:
[[  1.           0.        ]
 [  0.         -47.85416449]]
update_direction:
[[-2.83161263e-11]
 [ 2.96764769e-08]]
n=2, new x1:537664.0, new y1: 20.000000029069028,
information gain: -0.0002251793381955816, recall: 0.0449438202247191, filter rate: 0.11467537456780638
********************************
new_grads:
[[2.98908627e-11]
 [6.54438904e-10]]
hessian matrix:
[[ 1.00000000e+00 -4.59176146e-02]
 [-4.59176146e-02  8.65336192e+02]]
update_direction:
[[ 1.59410729e-13]
 [-5.66308297e-07]]
n=3, new x1:537664.0, new y1: 19.99999946276073,
information gain: -0.00020901150114722294, recall: 0.052434456928838954, filter rate: 0.12224356511717249
******

IndexError: positional indexers are out-of-bounds

In [ ]:
results

,iteration number,x1,y1,new x1,new y1,information gain,recall,filter rate
0,1.0,5.376357e+05,19.999393,5.376357e+05,19.999393,-0.000209,0.052434,0.122244
1,2.0,5.379137e+05,19.998772,5.379137e+05,19.998772,-0.000209,0.052434,0.122244
2,3.0,-2.291824e+15,5132.293204,-2.291824e+15,5132.293204,-0.000000,0.000000,0.000000


## 加入邊界條件
使用懲罰函數中的內點法，限制 x, y 必須為正，此時目標函數如下
$$min IG(x, y)$$
$$s.t. x\geq0\quad and\quad y\geq0$$
內點法的概念是初始點在可行區域內，建立起一道高牆，當接近邊界時懲罰會大增，使後續更新無法離開可行解的區域內，常見的內點法的逞罰函數會取倒數或log
$$min IG(x,y) + \gamma(ln(x) + ln(y))$$
另外x, y 上限受到training 資料影響，因為當越接近上限時，我們知道此時的資料幾乎會都分到 TO_SAM =0 的那類，filter rate 最大，但 recall 最低
2022/10/19

In [21]:
def gradientPena(point):
    if np.any(point < 0): # any 是只要有一個是 True 就 output True，但 all 是要全部都為 True 才 output True
        raise ValueError('point are negative!')
    return 1 / point

Hessain matrix 應該要是對稱矩陣，但我做出來了顯然不適

In [29]:
def BFGSWithPenalty(point0, x_name, y_name, data):
    results = pd.DataFrame(columns= ['iteration number', 'x1', 'y1', 'new x1', 'new y1', 'information gain', 'recall', 'filter rate'])
    d = 2
    gradIG = gradientIG(point0, x_name, y_name, data)
    grads = gradIG
    H = np.eye(d)
    print(f'Hessian matrix:\n{H}')
    point = point0
    n = 1
    
    
    
    while np.linalg.norm(grads) > 1e-50:
        if n > N_max:
            print(f'Maximum iteration reached!')
            break

        update_direction = -H @ grads
        #step_size = lineSearch(point, update_direction, grads, data)
        print(f'update_direction:\n{update_direction}')
        new_point = point + step_size * update_direction

        if np.any(new_point < 0):
            print('touch the boundary!')
            new_point[new_point < 0] = 0
            
        information_gain = informationGain(new_point, x_name, y_name, data)
        recall, filter_rate = RecallFilterRate(new_point, x_name, y_name, data)
        print(f'n={n}, new x1:{float(new_point[0])}, new y1: {float(new_point[1])},\ninformation gain: {information_gain}, recall: {recall}, filter rate: {filter_rate}')
        
        results = results.append({'iteration number': n, 'x1': float(point[0]), 'y1': float(point[1]), 'new x1': float(new_point[0]), \
                                 'new y1': float(new_point[1]), 'information gain': information_gain, 'recall': recall, \
                                 'filter rate': filter_rate}, ignore_index= True)        
        
        new_gradIG = gradientIG(new_point, x_name, y_name, data)
        #new_gardPenalty = gradientPena(new_point)
        new_grads = new_gradIG

        # if np.isnan(new_grads).all() == True:
        #     break
        #print(f'new_grads:{new_grads}')

        y = new_grads - grads
        #y = y.reshape((-1, 1))
        s = new_point - point
        denominator = 1 / (y.T @ s)
        li = np.eye(d) - (denominator * (s @ s.T))
        ri = np.eye(d) - (denominator * (y @ y.T))
        H = li @ H @ ri + (denominator * (s @ s.T))
        print(f'Hessian matrix:\n{H}')
        print(f'*'*32)
        gradIG = new_gradIG
        #gradientPenalty = new_gardPenalty
        grads = new_grads
        point = new_point       
        n += 1

    if np.linalg.norm(grads) < 1e-50:
        print('Early stopping!')
    
    return results

In [30]:
if __name__ == '__main__':
    np.random.seed(928)
    x1 = intialPoint('Credit_Amt', train_set)
    y1 = intialPoint('number_of_Credit', train_set)
    print(f'initial  point x1:{x1}, y1:{y1}')
    point = np.array([[x1], [y1]])
    results = BFGSWithPenalty(point, x_name, y_name, train_set)

initial  point x1:537664.0, y1:20.0
Hessian matrix:
[[1. 0.]
 [0. 1.]]
update_direction:
[[-2.83161263e-11]
 [-6.07450306e-10]]
n=1, new x1:537635.6838737424, new y1: 19.99939254969441,
information gain: -0.00020901150114722294, recall: 0.052434456928838954, filter rate: 0.12224356511717249
Hessian matrix:
[[ 2.00000000e+00 -4.48292177e-01]
 [ 2.14539468e-05  9.99990383e-01]]
********************************
update_direction:
[[ 2.78005740e-10]
 [-6.20138118e-10]]
n=2, new x1:537913.6896141628, new y1: 19.99877241157689,
information gain: -0.00020901150114722294, recall: 0.052434456928838954, filter rate: 0.12224356511717249
Hessian matrix:
[[-9.24380072e+12  4.14392355e+12]
 [ 2.06198375e+07 -9.24371083e+06]]
********************************
update_direction:
[[-2.29182392e+03]
 [ 5.11229443e-03]]
touch the boundary!
n=3, new x1:0.0, new y1: 5132.2932042199545,
information gain: -0.0, recall: 0.0, filter rate: 0.0
Hessian matrix:
[[ 2.05630954e+29 -9.21827478e+28]
 [-1.95430233e+27  8

In [50]:
# def BFGSWithPenalty(point0, x_name, y_name, data):
#     results = pd.DataFrame(columns= ['iteration number', 'x1', 'y1', 'new x1', 'new y1', 'information gain', 'recall', 'filter rate'])
#     d = 2
#     gradIG = gradientIG(point0, x_name, y_name, data)
#     gradPenalty = gradientPena(point0)
#     grads = gradIG + (gradPenalty * gamma)
#     H = np.eye(d)
#     print(f'Hessian matrix:\n{H}')
#     point = point0
#     n = 1
    
    
    
#     while np.linalg.norm(grads) > 1e-50:
#         if n > N_max:
#             print(f'Maximum iteration reached!')
#             break

#         update_direction = -H @ grads
#         #step_size = lineSearch(point, update_direction, grads, data)
#         print(f'update_direction:\n{update_direction}')
#         new_point = point + step_size * update_direction

#         if np.any(new_point < 0):
#             print('touch the boundary!')
#             new_point[new_point < 0] = 0.00000001
            
#         information_gain = informationGain(new_point, x_name, y_name, data)
#         recall, filter_rate = RecallFilterRate(new_point, x_name, y_name, data)
#         print(f'n={n}, new x1:{float(new_point[0])}, new y1: {float(new_point[1])},\ninformation gain: {information_gain}, recall: {recall}, filter rate: {filter_rate}')
        
#         results = results.append({'iteration number': n, 'x1': float(point[0]), 'y1': float(point[1]), 'new x1': float(new_point[0]), \
#                                  'new y1': float(new_point[1]), 'information gain': information_gain, 'recall': recall, \
#                                  'filter rate': filter_rate}, ignore_index= True)        
#         new_gradIG = gradientIG(new_point, x_name, y_name, data)
#         new_gardPenalty = gradientPena(new_point)
#         new_grads = new_gradIG + (new_gardPenalty * gamma)

#         # if np.isnan(new_grads).all() == True:
#         #     break
#         #print(f'new_grads:{new_grads}')

#         y = new_grads - grads
#         y = y.reshape((-1, 1))
#         s = new_point - point
#         denominator = 1 / (y.T @ s)
#         li = np.eye(d) - (denominator * (s @ s.T))
#         ri = np.eye(d) - (denominator * (y @ y.T))
#         H = li @ H @ ri + (denominator * (s @ s.T))
#         print(f'Hessian matrix:\n{H}')
#         print(f'*'*32)
#         gradIG = new_gradIG
#         gradientPenalty = new_gardPenalty
#         grads = new_grads
#         point = new_point       
#         n += 1

#     if np.linalg.norm(grads) < 1e-50:
#         print('Early stopping!')
    
#     return resultsa

In [45]:
#hyperparameter
SAR_type = 'TO_SAR'
eps = 1e-1000
N_max = 100
x_name = 'Credit_Amt'
y_name = 'number_of_Credit'
#step_size = np.array([[1e12], [1e6]])
step_size = 1
gamma = 1
auxiliary_point_select_behind_x = 3
auxiliary_point_select_behind_y = 1
father_entropy = Entropy(train_set, SAR_type)

In [51]:
if __name__ == '__main__':
    np.random.seed(928)
    x1 = intialPoint('Credit_Amt', train_set)
    y1 = intialPoint('number_of_Credit', train_set)
    print(f'initial  point x1:{x1}, y1:{y1}')
    point = np.array([[x1], [y1]])
    results = BFGSWithPenalty(point, x_name, y_name, train_set)

initial  point x1:537664.0, y1:20.0
Hessian matrix:
[[1. 0.]
 [0. 1.]]
update_direction:
[[-1.85992595e-06]
 [-5.00000006e-02]]
n=1, new x1:537663.99999814, new y1: 19.94999999939255,
information gain: -0.00020901150114722294, recall: 0.052434456928838954, filter rate: 0.12224356511717249
Hessian matrix:
[[ 1.00000000e+00  3.71987852e-05]
 [-2.26528933e-07  2.00250627e+00]]
********************************
update_direction:
[[-3.72449841e-06]
 [-1.00376255e-01]]
n=2, new x1:537663.9999944156, new y1: 19.849623744216917,
information gain: -0.00020901150114722294, recall: 0.052434456928838954, filter rate: 0.12224356511717249
Hessian matrix:
[[ 9.99999997e-01  1.48420589e-02]
 [-8.99319850e-05  4.01002545e+02]]
********************************
update_direction:
[[-7.49584857e-04]
 [-2.02020227e+01]]
touch the boundary!
n=3, new x1:537663.9992448307, new y1: 1e-08,
information gain: -0.00013465214908200962, recall: 0.4344569288389513, filter rate: 0.5295044179792547
Hessian matrix:
[[ 9.9

IndexError: index -3 is out of bounds for axis 0 with size 0

In [23]:
results

,iteration number,x1,y1,new x1,new y1,information gain,recall,filter rate
0,1.0,537664.000000,20.00,537663.999998,19.950000,-0.000209,0.052434,0.122244
1,2.0,537663.999998,19.95,537663.999994,19.849624,-0.000209,0.052434,0.122244


因為逞罰函數是 ln function，值域界在 $[0, \infty]$，所以上面結果x1, y1 ，不應該出現負的但還可以繼續執行，那是因為我是直接用 ln 的微分，所以避免掉值域區域的問題，所以我在計算微分之前多一個判斷檢查

# 考慮recall
老師提議的方式是利用 sigmoid 代替 indicator function
$$min L(x, y)=-IG(x, y) \times \sigma(recall(x,y))$$
將 sigmoid 平移到$\sigma(0.8)\approx 1$ <br>
透過 product rule and chain rule，我們可以得到新的目標函數的一階微分為
$$\nabla L(x, y)= \nabla -IG(x, y)\times \sigma(recall(x, y)) + -IG(x, y)\times \sigma^{\prime}(recall(x, y)) \times \nabla recall(x,y)$$

In [266]:
def gradientRecall(point, x_name, y_name, data):
    x1 = float(point[0])
    y1 = float(point[1])
    
    if pd.DataFrame(np.unique(data[data[x_name] > x1][x_name])).shape[0] < auxiliary_point_select_behind_x or \
        pd.DataFrame(np.unique(data[data[y_name] > y1][y_name])).shape[0] < auxiliary_point_select_behind_y:
        print('auxiliary point can not been found!')
        
        return np.array([[np.nan], [np.nan]])
    
    x2 = pd.DataFrame(np.unique(data[data[x_name] > x1][x_name])).sort_values(by= 0).iloc[auxiliary_point_select_behind_x, 0]
    y2 = pd.DataFrame(np.unique(data[data[y_name] > y1][y_name])).sort_values(by= 0).iloc[auxiliary_point_select_behind_y, 0]
    
    delta_x = x2 - x1
    delta_y = y2 - y1
    
    z = Recall(point, x_name, y_name, data)
    z_delta_x = Recall(np.array([[x2], [y1]]), x_name, y_name, data)
    z_delta_y = Recall(np.array([[x1], [x2]]), x_name, y_name, data)
    
    grad = np.array([[(z_delta_x - z) / delta_x], [(z_delta_y - z) / delta_y]])
    return grad

In [265]:
def sigmoid(x):
    return 1.0 / (1 + np.exp(-(sigmoid_b + sigmoid_w * x)))

In [258]:
def sigmoidDiff(x):
    tmp = sigmoid(x)
    return tmp * (1 - tmp)

In [276]:
def BFGSWithRecall(point0, x_name, y_name, data):
    results = pd.DataFrame(columns= ['iteration number', 'x1', 'y1', 'new x1', 'new y1', \
                                     'information gain', 'recall', 'filter rate'])
    d = 2
    gradIG = gradientIG(point0, x_name, y_name, data)
    gradRecall  = gradientRecall(point0, x_name, y_name, data)
    recall = Recall(point0, x_name, y_name, data)
    information_gain = informationGain(point0, x_name, y_name, data)
    grads = gradIG * sigmoid(recall) + information_gain * sigmoidDiff(recall) * gradRecall
    
    H = np.eye(d)
    point = point0
    n = 0
    
    
    
    while np.linalg.norm(grads) > 1e-50:
        if n > N_max:
            print(f'Maximum iteration reached!')
            break
        
        n += 1
        update_direction = -H @ grads
        #step_size = lineSearch(point, update_direction, grads, data)
        print(f'update_direction:{update_direction}')
        penatly_grad = np.array([[gradientPenalty(float(point[0]))], [gradientPenalty(float(point[1]))]])
        print(f'penatly_direction:{penatly_grad}')
        new_point = point + (step_size * update_direction) + (gamma * penatly_grad)

        information_gain = informationGain(point, x_name, y_name, data)
        recall, filter_rate = RecallFilterRate(point, x_name, y_name, data)
        
        print(f'n={n}, new x1:{point[0]}, new y1: {point[1]},\ninformation gain: {information_gain}, recall: {recall}, filter rate: {filter_rate}')
        
        results = results.append({'iteration number': n, 'x1': point[0], 'y1': point[1], 'new x1': new_point[0], \
                                 'new y1': new_point[1], 'information gain': information_gain, 'recall': recall, \
                                 'filter rate': filter_rate}, ignore_index= True)        
        
        new_gradIG = gradientIG(new_point, x_name, y_name, data)
        new_gradRecall = gradientRecall(new_point, x_name, y_name, data)
        
        if np.isnan(new_gradIG).all() == True or np.isnan(new_gradRecall).all() == True:
            break
        print(f'new_gradIG:{new_gradIG}')
        print(f'new_gradRecall:{new_gradRecall}')
        
        new_grads = new_gradIG * sigmoid(recall) + information_gain * sigmoidDiff(recall) * gradRecall
        print(f'new_grads:{new_gradIG}')
        
        y = new_grads - grads
        y = y.reshape((-1, 1))
        s = new_point - point
        denominator = 1 / (y.T @ s)
        li = np.eye(d) - (denominator * (s @ s.T))
        ri = np.eye(d) - (denominator * (y @ y.T))
        H = li @ H @ ri + (denominator * (s @ s.T))
        grads = new_grads
        point = new_point
    
    if np.linalg.norm(grads) < 1e-50:
        print('Early stopping!')
    
    return results

In [268]:
#hyperparameter
SAR_type = 'TO_SAR'
eps = 1e-1000
N_max = 100
x_name = 'Credit_Amt'
y_name = 'number_of_Credit'
#learning_rate = 1e-100
# x_array = np.unique(train_set[x_name].dropna())
# y_array = np.unique(train_set[y_name].dropna())
# number_x = 100
# number_y = 100
#step_size = np.array([[1e12], [1e6]])
auxiliary_point_select_behind_x = 3
auxiliary_point_select_behind_y = 3
step_size = 1
gamma = 1
sigmoid_b = 0.8
sigmoid_w = 5
father_entropy = Entropy(train_set, SAR_type)

In [278]:
if __name__ == '__main__':
    np.random.seed(928)
    x1 = intialPoint('Credit_Amt', train_set)
    y1 = intialPoint('number_of_Credit', train_set)
    print(f'initial  point x1:{x1}, y1:{y1}')
    point = np.array([[x1], [y1]])
    results = BFGSWithRecall(point, x_name, y_name, train_set)

initial  point x1:537664.0, y1:20.0
update_direction:[[-1.16157404e-11]
 [-3.80679673e-06]]
penatly_direction:[[1.85989763e-06]
 [5.00000000e-02]]
n=1, new x1:[537664.], new y1: [20.],
information gain: -8.846741616871934e-05, recall: 0.0749063670411985, filter rate: 0.1146731718560919
new_gradIG:[[1.70541202e-11]
 [1.10223572e-06]]
new_gradRecall:[[0.        ]
 [0.01896362]]
new_grads:[[1.70541202e-11]
 [1.10223572e-06]]
update_direction:[[-3.32403876e-11]
 [-1.08668929e-06]]
penatly_direction:[[1.85989763e-06]
 [4.98753212e-02]]
n=2, new x1:[537664.00000186], new y1: [20.04999619],
information gain: -6.502468907649411e-05, recall: 0.0749063670411985, filter rate: 0.1146731718560919
new_gradIG:[[1.70541203e-11]
 [1.11633094e-06]]
new_gradRecall:[[0.        ]
 [0.01920612]]
new_grads:[[1.70541203e-11]
 [1.11633094e-06]]
update_direction:[[1.30971394e-05]
 [3.51215880e-01]]
penatly_direction:[[1.85989763e-06]
 [4.97515645e-02]]
n=3, new x1:[537664.00000372], new y1: [20.09987043],
infor

IndexError: index 3 is out of bounds for axis 0 with size 0

## 加入上限的逞罰條件
2022/10/30
加入上限的逞罰條件，避免閥值更新超過上限 <br>
另外就是會遇到接近上限時會找不到輔助點計算斜率，現在是找大於閥值後的第$n$個數，就會變成找小於閥值的第$n$個數